# Geração de scripts de TV
Neste projeto, você gerará seus próprios scripts de TV [Simpsons] (https://en.wikipedia.org/wiki/The_Simpsons) usando RNNs. Você estará usando parte do [conjunto de dados Simpsons] (https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) de scripts de 27 temporadas. A Rede Neural que você irá construir irá gerar um novo roteiro de TV para uma cena em [Moe's Tavern] (https://simpsonswiki.com/wiki/Moe's_Tavern).
## Obtém os dados
Os dados já são fornecidos para você. Você estará usando um subconjunto do conjunto de dados original. Consiste apenas nas cenas da Taverna de Moe. Isso não inclui outras versões da taverna, como "Moe's Cavern", "Flame Moe's", "Uncle Moe's Family Feed-Bag", etc.

In [16]:
import tensorflow as tf
tf.__version__

'1.1.0'

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [3]:
!cat requirements.txt | grep numpy 

numpy==1.12.1


In [9]:
!pip3 install --upgrade numpy

  Using cached https://files.pythonhosted.org/packages/f5/bf/4981bcbee43934f0adb8f764a1e70ab0ee5a448f6505bd04a87a2fda2a8b/numpy-1.16.1-cp36-cp36m-manylinux1_x86_64.whl


In [14]:
#!pip3 uninstall numpy

In [18]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implementar funções de pré-processamento
A primeira coisa a fazer em qualquer conjunto de dados é o pré-processamento. Implemente as seguintes funções de pré-processamento abaixo:
- Tabela de pesquisa
- Tokenize pontuação

### Tabela de pesquisa
Para criar uma incorporação de palavras, você primeiro precisa transformar as palavras em ids. Nesta função, crie dois dicionários:
- Dicionário para ir das palavras para um id, vamos chamar `vocab_to_int`
- Dicionário para ir do id para a palavra, vamos chamar `int_to_vocab`

Devolve estes dicionários na seguinte tupla `(vocab_to_int, int_to_vocab)`

In [19]:
import numpy as np
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # Implement Function
    cnts = Counter(text)
    vocab = sorted(cnts, key=cnts.get, reverse=True)
    vocab_to_int = {word: ix for ix, word in enumerate(vocab)}
    int_to_vocab = {ix: word for word, ix in vocab_to_int.items()}
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
Estaremos dividindo o script em um array de palavras usando espaços como delimitadores. No entanto, pontuações como períodos e pontos de exclamação tornam difícil para a rede neural distinguir entre a palavra "bye" e "bye!".

Implemente a função `token_lookup` para retornar um comando que será usado para tokenizar símbolos como"! " em "|| Exclamation_Mark ||". Crie um dicionário para os seguintes símbolos, onde o símbolo é a chave e o valor é o token:
- Período (.)
- vírgula (,)
- Aspa ( " )
- ponto e vírgula (;)
- Ponto de exclamação ( ! )
- Ponto de interrogação ( ? )
- Parêntese esquerdo (()
- Parênteses direitos ())
- traço (-)
- retorno (\ n)

Este dicionário será usado para simbolizar os símbolos e adicionar o delimitador (espaço) ao redor dele. Isso separa os símbolos como sua própria palavra, facilitando a previsão da rede neural na próxima palavra. Certifique-se de não usar um token que possa ser confundido como uma palavra. Em vez de usar o token "dash", tente usar algo como "|| dash ||".

In [20]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    return {'.':'||Period||',
            ',':'||Comma||',
            '"':'||Quotation_Mark||',
            ';':'||Semicolon||', 
            '!':'||Exclamation_Mark||',
            '?':'||Question_Mark||', 
            '(':'||Left_Parentheses||', 
            ')':'||Right_Parentheses||',
            '--':'||Dash||',
            '\n':'||Return||'}

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Pré-processe todos os dados e salve-os
A execução da célula de código abaixo irá pré-processar todos os dados e salvá-los no arquivo.

In [21]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
Este é seu primeiro checkpoint. Se você decidir voltar a este notebook ou precisar reiniciar o notebook, poderá começar a partir daqui. Os dados pré-processados ​​foram salvos no disco.

In [23]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Construa a Rede Neural
Você construirá os componentes necessários para construir um RNN implementando as seguintes funções abaixo:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Verifique a versão do TensorFlow e acesse o GPU

In [24]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0


/home/joana/.local/lib/python3.6/site-packages/ipykernel/__main__.py:14: UserWarning: No GPU found. Please use a GPU to train your neural network.


### Entrada
Implementar o `
get_inputs () `para criar espaços reservados para TF para a rede neural. Deve criar os seguintes marcadores de posição:
- Espaço reservado para entrada de texto chamado "input" usando o parâmetro [TF Placeholder] (https://www.tensorflow.org/api_docs/python/tf/placeholder) `name`.
- Espaço reservado para alvos
- Espaço reservado para Learning Rate

Retorna os placeholders na seguinte tupla `(Input, Targets, LearningRate)`

In [25]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # Implement Function
    Entrada = tf.placeholder(tf.int32, [None, None], name='input')
    Alvo = tf.placeholder(tf.int32, [None, None], name='targets')
    NivelDeAprendizado = tf.placeholder(tf.float32, name='learning_rate')
    return Entrada, Alvo, NivelDeAprendizado


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Construir Célula RNN e Inicializar
Empilhe uma ou mais [`BasicLSTMCells`] (https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) em um [` MultiRNNCell`] (https://www.tensorflow.org/ api_docs / python / tf / contrib / rnn / MultiRNNCell).
- O tamanho do Rnn deve ser definido usando `rnn_size`
- Initalize Cell State usando a função [`zero_state ()`] (https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) do MultiRNNCell
    - Aplique o nome "initial_state" ao estado inicial usando [`tf.identity ()`] (https://www.tensorflow.org/api_docs/python/tf/identity)

Retorna a célula e o estado inicial na seguinte tupla `(Cell, InitialState)`

In [26]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    ## definindo LSTM e CELULA
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    cell = tf.contrib.rnn.MultiRNNCell([lstm] * 1) 
    
    ## estagio zero do rolé e estado inicial
    zero_state = cell.zero_state(batch_size, tf.float32)
    initial_state = tf.identity(zero_state, name="initial_state")
    
    return cell, initial_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Incorporação de palavras
Aplique a incorporação ao `input_data` usando o TensorFlow. Retornar a seqüência incorporada.

In [29]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # Implement Function
    embedding = tf.Variable(tf.random_uniform([vocab_size, embed_dim], -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Construir RNN
Você criou uma célula RNN na função `get_init_cell ()`. Hora de usar a célula para criar um RNN.
- Construa o RNN usando o [`tf.nn.dynamic_rnn ()`] (https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Aplique o nome "final_state" ao estado final usando [`tf.identity ()`] (https://www.tensorflow.org/api_docs/python/tf/identity)

Retorna as saídas e o estado final_state na seguinte tupla `(Saídas, FinalState)`

In [32]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    Outputs, Final_State = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    Final_State = tf.identity(Final_State, name='final_state')
    return Outputs, Final_State


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Construa a Rede Neural
Aplique as funções que você implementou acima para:
- Aplicar incorporação ao `input_data` usando sua função` get_embed (input_data, vocab_size, embed_dim) `.
- Construa RNN usando `cell` e sua função` build_rnn (cell, inputs) `.
- Aplique uma camada conectada _com uma ativação _linear_ e `vocab_size` como o número de saídas.

Retorna os logits e o estado final na seguinte tupla (Logits, FinalState)

In [34]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # Implement Function
    embed = get_embed(input_data, vocab_size, embed_dim)
    Output, Final_State = build_rnn(cell, embed)
    Logits = tf.contrib.layers.fully_connected(Output, vocab_size, activation_fn=None)
    return Logits, Final_State


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Lotes
Implemente `get_batches` para criar lotes de entrada e de destino usando` int_text`. Os lotes devem ser um array Numpy com a forma `(número de lotes, 2, tamanho do lote, comprimento da seqüência)`. Cada lote contém dois elementos:
- O primeiro elemento é um único lote de ** entrada ** com a forma `[tamanho do lote, comprimento da sequência]`
- O segundo elemento é um único lote de ** destinos ** com a forma `[tamanho do lote, comprimento da sequência]`

Se você não puder preencher o último lote com dados suficientes, elimine o último lote.

Por exemplo, `get_batches ([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2) `retornaria uma matriz Numpy dos seguintes:
`` `
[
  # Primeiro lote
  [
    # Lote de entrada
    [[1 2], [7 8], [13 14]]
    # Lote de alvos
    [[2 3], [8 9], [14 15]]
  ]

  # Segundo lote
  [
    # Lote de entrada
    [[3 4], [9 10], [15 16]]
    # Lote de alvos
    [[4 5], [10 11], [16 17]]
  ]

  # Terceiro lote
  [
    # Lote de entrada
    [[5 6], [11 12], [17 18]]
    # Lote de alvos
    [[6 7], [12 13], [18 1]]
  ]
]
`` `

Observe que o último valor de destino no último lote é o primeiro valor de entrada do primeiro lote. Neste caso, `1`. Essa é uma técnica comum usada na criação de lotes de sequências, embora não seja intuitiva.

In [47]:
def get_batches(int_text, batch_size, seq_length):
    Chunk_Size = batch_size * seq_length
    Numbers_Chunks = len(int_text) // Chunk_Size
    if (Numbers_Chunks*Chunk_Size==len(int_text)): int_text = int_text + int_text[0:1]
    else: int_text[Numbers_Chunks*Chunk_Size] = int_text[0]
    si = [i for i in range(0, len(int_text)-seq_length+1, seq_length)]
    batches = []
    for i in range(Numbers_Chunks):
        x = [] 
        y = [] 
        for j in range(batch_size):
            start = si[j*Numbers_Chunks + i]
            x.append(int_text[start:start+seq_length])
            start += 1
            y.append(int_text[start:start+seq_length])
        batches.append([x, y])
    return np.array(batches)

In [48]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [65]:
# Number of Epochs
num_epochs = 50
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 1024
# Sequence Length
seq_length = 16
# Learning Rate
learning_rate = 0.001
# Show stats for every n number of batches
show_every_n_batches = 11

"""DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [66]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [67]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/33   train_loss = 8.823
Epoch   0 Batch   11/33   train_loss = 6.545
Epoch   0 Batch   22/33   train_loss = 6.481
Epoch   1 Batch    0/33   train_loss = 6.029
Epoch   1 Batch   11/33   train_loss = 5.694
Epoch   1 Batch   22/33   train_loss = 5.838
Epoch   2 Batch    0/33   train_loss = 5.586
Epoch   2 Batch   11/33   train_loss = 5.367
Epoch   2 Batch   22/33   train_loss = 5.555
Epoch   3 Batch    0/33   train_loss = 5.305
Epoch   3 Batch   11/33   train_loss = 5.129
Epoch   3 Batch   22/33   train_loss = 5.334
Epoch   4 Batch    0/33   train_loss = 5.105
Epoch   4 Batch   11/33   train_loss = 4.965
Epoch   4 Batch   22/33   train_loss = 5.157
Epoch   5 Batch    0/33   train_loss = 4.929
Epoch   5 Batch   11/33   train_loss = 4.804
Epoch   5 Batch   22/33   train_loss = 4.969
Epoch   6 Batch    0/33   train_loss = 4.772
Epoch   6 Batch   11/33   train_loss = 4.647
Epoch   6 Batch   22/33   train_loss = 4.812
Epoch   7 Batch    0/33   train_loss = 4.609
Epoch   7 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [68]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [69]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

In [70]:
## TIVE MT PROBLEMA COM O RAIO DA VERSAO DO TENSORFLOW
print (tf.__version__)

1.1.0


## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [71]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # Implement Function
    InputTensor = loaded_graph.get_tensor_by_name("input:0")
    InitialStateTensor = loaded_graph.get_tensor_by_name("initial_state:0")
    FinalStateTensor = loaded_graph.get_tensor_by_name("final_state:0")
    ProbsTensor = loaded_graph.get_tensor_by_name("probs:0")
    return InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Escolher palavra
Implemente a função `pick_word ()` para selecionar a próxima palavra usando `probabilities`.

In [72]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    idx = np.argmax(probabilities)
    return int_to_vocab[idx]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)



Tests Passed


## Gerar script de TV
Isso irá gerar o script de TV para você. Defina `gen_length` com o comprimento do script de TV que você deseja gerar.

In [ ]:
gen_length = 300
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
moe_szyslak:(defeated) okay, homer, i got somethin'.
barney_gumble:(consulting a big wave)
moe_szyslak: what's the matter, moe?
homer_simpson:(leans back, thoughtfully) well, i'll take these, but i could whip, but i bet they'd have a kid like me.
moe_szyslak: well, somebody's a grumpy gus. what's the matter, homer?
moe_szyslak: ah, gee, it ain't no mystery-- he don't know.
carny: teacup? how'd that pretty bad friend.
moe_szyslak: hey, hey, hey, hey, hey, hey, hey! don't you badmouth this country. compared to me!
moe_szyslak: ah, i'm a little busy, i just talk about my beer.
moe_szyslak:(musses bart's hair) so, that's a good one, moe. i got a big payday(grabs beer)
homer_simpson:(grumpy) yeah, i can't.
carl_carlson: no, that's just gasoline and nauseous, but i ain't angry.
carl_carlson:(backing away) no, you're a published author!
moe_szyslak:(wiping) oh, that's it... homer, i was just drawin' wangs on the numbers.
lenny_leonard: yeah, th

# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the beggining of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.